In [13]:
import dataprep
import pandas as pd

import numpy as np
from scipy import sparse as sp
from scipy.sparse.linalg import norm
import sklearn.preprocessing as pp

from sklearn import svm
from sklearn.metrics import precision_score

In [34]:
used_features = ["engaging_user_id", "tweet_id"]
target_features = ["retweet"]
# reply, like, retweet_with_comment
data = dataprep.import_data(source_features=used_features, target_features=target_features, nrows=500000)
data = data[data["retweet"] == 1]

In [41]:
tweetIds = data["tweet_id"].unique()
tweetIds.sort()
userIds = data["engaging_user_id"].unique()
userIds.sort()

m = userIds.size
n = tweetIds.size
numRatings = len(data)

In [75]:
test =[1,2,3] + [6,7]
test

[1, 2, 3, 6, 7]

In [84]:
data[data["retweet"] == 1]

data[data["tweet_id"] == "930A87A198DDD42CB5BEEE76FF645B45"]

data[data["engaging_user_id"] == "45BD806DD458963F76077A1E5D9F340E"]

dict.fromkeys(data)

user_ratings = {}
tweets_users = {}

for row_id in range(0, len(data)):
    row = data.iloc[row_id]
    if row["engaging_user_id"] in user_ratings:
        user_ratings[row["engaging_user_id"]].append(row["tweet_id"])
    else:
        user_ratings[row["engaging_user_id"]] = [row["tweet_id"]]
        
    if row["tweet_id"] in tweets_users:
        tweets_users[row["tweet_id"]].append(row["engaging_user_id"])
    else:
        tweets_users[row["tweet_id"]] = [row["engaging_user_id"]]
        
user_ratings
    


{'7A4AADFBF8BAEE8874F05E5CEA149B4E': ['4B3937ED6F4F26E61914BA282E1FB509'],
 '45BD806DD458963F76077A1E5D9F340E': ['873FBA2FD3A09BF98CC6D0EB84C0E2B2'],
 '1958C5A1AF21AE7DFA1B709F5D45BB9C': ['A8AC72AD1B865DD430512FBDE6E2FA4B',
  '174ECD666A72A8AFE47A890C88182452'],
 '5D2073E67B1B5100EF1BE49DCCAF607F': ['A8AC72AD1B865DD430512FBDE6E2FA4B'],
 'BEA9E15A048EDC7573857062E4E04B2E': ['A8AC72AD1B865DD430512FBDE6E2FA4B',
  '174ECD666A72A8AFE47A890C88182452'],
 '56A6F62DA0294E5224217F0C9D511D1E': ['B750B9F07B7C364B5321C9D80279B137'],
 '282A24F47AC8E10AA9376358EF9130FB': ['6031935454DF5F99D9C41A8C684D9F0B'],
 '189E86199FFB92E7ED909F1FC8DF76F7': ['DCEF6C06DDE77C2DBE7F0BE99B95120A'],
 '2284A3F835F7156B2F432B82D8963D27': ['DCEF6C06DDE77C2DBE7F0BE99B95120A'],
 '27F599ACAF4BC138E6A55F55713F9133': ['DCEF6C06DDE77C2DBE7F0BE99B95120A'],
 '4DEEE843E2B203B58A3C7837982C5516': ['DCEF6C06DDE77C2DBE7F0BE99B95120A'],
 'AF2FC6240EB5DDE269197A1D777D6D22': ['DCEF6C06DDE77C2DBE7F0BE99B95120A'],
 '7E8647858EB1956C4F2ABD

In [85]:
#R = sp.csr_matrix((data.retweet, (userints, tweetints)))
#R_dok = R.todok()

In [78]:
def compute_pairwise_user_similarity(u_id, v_id):
    u = data[data["engaging_user_id"] == u_id].copy()
    v = data[data["engaging_user_id"] == v_id].copy()
    
    # using the formula on slide 25 of slide deck 2

    
    # calculate sqrt of sum of (r_ui - mean(r_u))^2
    u_denominator = np.sqrt(sum(u["retweet"]))
    
    # calculate sqrt of sum of (r_vi - mean(r_v))^2
    v_denominator = np.sqrt(sum(v["retweet"]))
    
    denominator = u_denominator * v_denominator
    
    
    # erstelle liste aus item ids für beide user u<>v
    items = user_ratings[u_id] + user_ratings[v_id]
    items = set(items)
    
    #iteriere über list
    
    
    numerator = 0
    def calculate_nth_summand(item):
        if item in user_ratings[u_id] and item in user_ratings[v_id]:
            return 1
        else:
            return 0
        
    numerator = np.array([calculate_nth_summand(i) for i in items]).sum()
         
    
    if denominator == 0:
        similarity = 0.;
    else:
        similarity = numerator/denominator
    
    return similarity

In [81]:
compute_pairwise_user_similarity("1958C5A1AF21AE7DFA1B709F5D45BB9C", "5D2073E67B1B5100EF1BE49DCCAF607F")

0.7071067811865475

In [82]:
def compute_user_similarities(u_id):
    '''
    Much faster matrix-based approach
    '''
    
    similarities = []
    
    for userId in userIds:
        similarities.append(compute_pairwise_user_similarity(u_id, userId))

    
    
    return similarities

In [83]:
compute_user_similarities("1958C5A1AF21AE7DFA1B709F5D45BB9C")

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [ ]:
## default values
k = 5

def create_user_neighborhood(u_id, i_id):
    nh = {} ## the neighborhood dict with (user id: similarity) entries
    ## nh should not contain u_id and only include users that have rated i_id; there should be at most k neighbors
    uU = compute_user_similarities(u_id)
    uU_copy = uU.copy() ## so that we can modify it, but also keep the original
    
    sort_index = np.flip(np.argsort(uU_copy))
    
    taken = 0
    for i in sort_index:
        if i == u_id:
            continue
        if (i, i_id) in R_dok:
            nh[i] = uU_copy[i]
            taken += 1
            if (taken >= k):
                break
    
    return nh

In [ ]:
## a default value
with_deviations = True

def predict_rating(u_id, i_id):
    
    if (u_id, i_id) in R_dok:
        print("user", u_id, "has rated item", i_id, "with", R[u_id, i_id])
    else:
        print("user", u_id, "has not rated item", i_id)
    print("k:", k, "with_deviations:", with_deviations, "with_abs_sim:", with_abs_sim)
    
    
    nh = create_user_neighborhood(u_id, i_id)
    
    neighborhood_weighted_avg = 0.
    
    if with_deviations:
        numerator = np.array([nh[x] * (R[x, i_id] - user_avgs[x]) for x in nh]).sum()

    else:
        numerator = np.array([nh[x] * (R[x, i_id]) for x in nh]).sum()

    denominator = np.array([abs(nh[x]) for x in nh]).sum()
    
    neighborhood_weighted_avg = numerator / denominator
    
    
    if with_deviations:
        prediction = user_avgs[u_id] + neighborhood_weighted_avg
        print(f'prediction {prediction:.4f} (user_avg {user_avgs[u_id]:.4f} offset {neighborhood_weighted_avg:.4f})')
    else:
        prediction = neighborhood_weighted_avg
        print(f'prediction {prediction:.4f} (user_avg {user_avgs[u_id]:.4f})')
        
    return prediction